In [1]:
import folium
import pandas as pd
import re
from shapely.geometry import Polygon
import geopandas as gpd
import numpy as np
import ast

In [2]:
LEZ_location = pd.read_csv(r"..\..\data\ProcessedData\NDW_locations.csv")
LEZ_location = LEZ_location[LEZ_location['zone_name'].str.startswith('ZE') == False].reset_index(drop=True)
LEZ_location = LEZ_location[LEZ_location['zone_name'] != "Milieuzone Amsterdam"].reset_index(drop=True)

def parse_coords(coord_str):
    # Clean and parse string into (lat, lon) pairs list
    coord_str = coord_str.strip("[]'\" ")
    coords_split = coord_str.split()
    coords_clean = [re.sub(r"[,'\"]", "", c) for c in coords_split]
    latlons = [(float(coords_clean[i]), float(coords_clean[i+1])) for i in range(0, len(coords_clean), 2)]
    return latlons

# Apply function to the column 'coordinate_sets', storing results in a new column 'latlon_pairs'
LEZ_location['latlon_pairs'] = LEZ_location['coordinate_sets'].apply(parse_coords)

LEZ_location.head()

,zone_name,coordinate_sets,start_date,end_date,latlon_pairs
0,LEZ 's-Hertogenbosch,['51.691127953 5.294504572 51.6886171294 5.293...,2020-01-01T00:00:00Z,2025-03-01T00:00:00Z,"[(51.691127953, 5.294504572), (51.6886171294, ..."
1,LEZ Delft,['52.013883 4.365207 52.013771 4.365007 52.013...,2020-01-01T00:00:00Z,2999-01-01T00:00:00Z,"[(52.013883, 4.365207), (52.013771, 4.365007),..."
2,LEZ Haarlem,['52.387318 4.650521 52.387734 4.64918 52.3883...,2020-01-01T00:00:00Z,2999-01-01T00:00:00Z,"[(52.387318, 4.650521), (52.387734, 4.64918), ..."
3,onbekend,['52.20791599788392 5.958141001309827 52.20938...,2025-01-01T00:00:00Z,2026-12-31T23:00:00Z,"[(52.20791599788392, 5.958141001309827), (52.2..."
4,LEZ Den Haag,['52.075427 4.329351 52.075976 4.329212 52.076...,2020-01-01T00:00:00Z,2999-01-01T00:00:00Z,"[(52.075427, 4.329351), (52.075976, 4.329212),..."


In [3]:
df = LEZ_location

df['LAT'] = df['latlon_pairs'].apply(lambda x: [lat for lon, lat in x])
df['LON'] = df['latlon_pairs'].apply(lambda x: [lon for lon, lat in x])

geom_list = [(x, y) for x, y  in zip(df['LON'],df['LAT'])]

geom_list_2 = [Polygon(tuple(zip(x, y))) for x, y in geom_list]

polygon_gdf =  gpd.GeoDataFrame(geometry=geom_list_2)

In [4]:
print(polygon_gdf)

                                             geometry
0   POLYGON ((51.69113 5.2945, 51.68862 5.29358, 5...
1   POLYGON ((52.01388 4.36521, 52.01377 4.36501, ...
2   POLYGON ((52.38732 4.65052, 52.38773 4.64918, ...
3   POLYGON ((52.20792 5.95814, 52.20939 5.95777, ...
4   POLYGON ((52.07543 4.32935, 52.07598 4.32921, ...
5   POLYGON ((51.57938 4.77048, 51.57938 4.77049, ...
6   POLYGON ((51.54421 5.05947, 51.54402 5.06125, ...
7   POLYGON ((52.09075 5.10505, 52.0904 5.10401, 5...
8   POLYGON ((52.16022 4.47762, 52.16082 4.48051, ...
9   POLYGON ((52.04555 4.33742, 52.04701 4.33954, ...
10  POLYGON ((51.98459 5.90506, 51.98476 5.9063, 5...
11  POLYGON ((52.39999 4.84996, 52.40012 4.85022, ...
12  POLYGON ((50.84966 5.67752, 50.84965 5.67797, ...
13  POLYGON ((51.9159 3.99302, 51.91756 3.99984, 5...


In [5]:
LEZ_location = pd.merge(LEZ_location, polygon_gdf, left_index=True, right_index=True)

In [6]:
print(LEZ_location)

                   zone_name  \
0       LEZ 's-Hertogenbosch   
1                  LEZ Delft   
2                LEZ Haarlem   
3                   onbekend   
4               LEZ Den Haag   
5                  LEZ Breda   
6                LEZ Tilburg   
7                LEZ Utrecht   
8                 LEZ Leiden   
9               LEZ Rijswijk   
10         Milieuzone Arnhem   
11             LEZ Amsterdam   
12     Milieuzone Maastricht   
13  LEZ Rotterdam Maasvlakte   

                                      coordinate_sets            start_date  \
0   ['51.691127953 5.294504572 51.6886171294 5.293...  2020-01-01T00:00:00Z   
1   ['52.013883 4.365207 52.013771 4.365007 52.013...  2020-01-01T00:00:00Z   
2   ['52.387318 4.650521 52.387734 4.64918 52.3883...  2020-01-01T00:00:00Z   
3   ['52.20791599788392 5.958141001309827 52.20938...  2025-01-01T00:00:00Z   
4   ['52.075427 4.329351 52.075976 4.329212 52.076...  2020-01-01T00:00:00Z   
5   ['51.579381 4.770481 51.579378 4.77049 51

In [7]:
center_lat = 52.2
center_lon = 5.3

m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=7,
        tiles='CartoDB Positron'
    )
m

In [8]:
for idx, row in LEZ_location.iterrows():

    folium.Polygon(
    locations=LEZ_location.latlon_pairs.iloc[idx],
    color="blue",
    weight=3,
    fill_color="red",
    fill_opacity=0.2,
    fill=True,
    popup= LEZ_location['zone_name'].iloc[idx],
    ).add_to(m)

m.save('test_map.html')